## Imports

In [ ]:
!pip install -q jupyter-dash
!pip install -q pyTigerGraph
!pip install -q dash-bootstrap-components
!pip install -q dash_daq
!pip install dash-extensions
!pip install swifter
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pyTigerGraph as tg
import pandas as pd
import dash_bootstrap_components as dbc
import datetime
from dash.exceptions import PreventUpdate
import plotly.figure_factory as ff
import dash_table
import json
from dash.dash import no_update
import dash_daq as daq
from dash_extensions import Download
from dash_extensions.snippets import send_data_frame
import swifter

## TigerGraph

### Connection

In [2]:
# Replace with your information
configs = {
    "host": "https://<YOUR_HOST>.i.tgcloud.io",
    "password": "tigergraph",
    "graphname": "AntiFraud"
}

In [3]:
# Instantiate connection to your TG server
conn = tg.TigerGraphConnection(host=configs['host'], password=configs['password'], gsqlVersion="3.0.5", useCert=True, graphname=configs['graphname'])
conn.apiToken = conn.getToken(conn.createSecret())

### Update Graph

In [ ]:
# Adds necessary vertices to create a time tree
# https://shreya-chaudhary.medium.com/creating-a-timetree-in-tigergraph-e71505724e77
print(conn.gsql('''
USE GRAPH AntiFraud
CREATE SCHEMA_CHANGE JOB addTimeTree FOR GRAPH AntiFraud {
    ADD VERTEX Year (PRIMARY_ID id INT, text STRING) WITH primary_id_as_attribute="true";
    ADD VERTEX Month (PRIMARY_ID id INT, text STRING) WITH primary_id_as_attribute="true";
    ADD VERTEX Day (PRIMARY_ID id INT, text STRING, dateValue DATETIME) WITH primary_id_as_attribute="true";
    ADD UNDIRECTED EDGE DAY_TO_TRANSACTION (FROM Day, TO Transaction);
    ADD UNDIRECTED EDGE MONTH_TO_DAY (FROM Month, TO Day);
    ADD UNDIRECTED EDGE YEAR_TO_MONTH (FROM Year, TO Month);
}
''', options=[]))

In [ ]:
# Runs job to add the time tree
print(conn.gsql('''
USE GRAPH AntiFraud
RUN SCHEMA_CHANGE JOB addTimeTree
''', options=[]))

### Queries

In [ ]:
# Populates time tree with data from graph 
print(conn.gsql('''
USE GRAPH AntiFraud
CREATE QUERY TransactionTimes() FOR GRAPH AntiFraud { 
  /* Inserts transaction time data into day, month, and year vertices in the graph */ 

  Seed = {Transaction.*};
  
  results = SELECT s FROM Seed:s
              ACCUM
                INSERT INTO Day (PRIMARY_ID, text, dateValue) VALUES (str_to_int(to_string(year(epoch_to_datetime(s.ts))) + to_string(month(epoch_to_datetime(s.ts))) + to_string(day(epoch_to_datetime(s.ts)))), to_string(day(epoch_to_datetime(s.ts))), epoch_to_datetime(s.ts)),
                INSERT INTO Month (PRIMARY_ID, text) VALUES (str_to_int(to_string(year(epoch_to_datetime(s.ts))) + to_string(month(epoch_to_datetime(s.ts)))), to_string(month(epoch_to_datetime(s.ts)))),
                INSERT INTO Year (PRIMARY_ID, text) VALUES (year(epoch_to_datetime(s.ts)), to_string(year(epoch_to_datetime(s.ts)))),
  
                INSERT INTO YEAR_TO_MONTH (FROM, TO) VALUES (year(epoch_to_datetime(s.ts)), str_to_int(to_string(year(epoch_to_datetime(s.ts))) + to_string(month(epoch_to_datetime(s.ts))))),
                INSERT INTO MONTH_TO_DAY (FROM, TO) VALUES (str_to_int(to_string(year(epoch_to_datetime(s.ts))) + to_string(month(epoch_to_datetime(s.ts)))), str_to_int(to_string(year(epoch_to_datetime(s.ts))) + to_string(month(epoch_to_datetime(s.ts))) + to_string(day(epoch_to_datetime(s.ts))))),
                INSERT INTO DAY_TO_TRANSACTION (FROM, TO) VALUES (str_to_int(to_string(year(epoch_to_datetime(s.ts))) + to_string(month(epoch_to_datetime(s.ts))) + to_string(day(epoch_to_datetime(s.ts)))), s);
}
INSTALL QUERY TransactionTimes
''', options=[]))

In [ ]:
# Returns most recent transaction after given timestamp (used for live updates to data table)
print(conn.gsql('''
USE GRAPH AntiFraud
CREATE QUERY GetMostRecentTransaction(INT timeStamp=1492214384, STRING dateStamp="2017-04-15", STRING sortValue = "") FOR GRAPH AntiFraud syntax V2{ 
  /* Gets transaction that happened most recently after the given timestamp */ 
  ListAccum<VERTEX> @receiverSet, @senderSet;
  SumAccum<FLoat> @receiverTrust, @senderTrust;
  Seed = {Day.*};
  
  IF sortValue == ">50" THEN
      s1 = SELECT t FROM Seed:d -(DAY_TO_TRANSACTION:e) -:t
           WHERE d.dateValue >= to_datetime(dateStamp) AND t.amount > 50;
  ELSE IF sortValue == "<=50" THEN
      s1 = SELECT t FROM Seed:d -(DAY_TO_TRANSACTION:e) -:t
           WHERE d.dateValue >= to_datetime(dateStamp) AND t.amount <= 50;
  ELSE 
      s1 = SELECT t FROM Seed:d -(DAY_TO_TRANSACTION:e) -:t
           WHERE d.dateValue >= to_datetime(dateStamp);
  END;
  
  IF sortValue == "<=.5" THEN
      s2 = SELECT t FROM s1:t - (User_Recieve_Transaction_Rev>:e1) - User:u1 -(User_Recieve_Transaction>:e2) -Transaction:x -(User_Transfer_Transaction_Rev>:e3) - User:u2
  
          WHERE u1.trust_score < 0.5 AND u2.trust_score < 0.5 AND t.ts > timeStamp
          ACCUM
            t.@receiverSet = u1,
            t.@receiverTrust = u1.trust_score,
            t.@senderSet = u2,
            t.@senderTrust = u2.trust_score
          
          ORDER BY t.ts ASC
          LIMIT 1;
  ELSE IF sortValue == ">.5" THEN
  
      s2 = SELECT t FROM s1:t - (User_Recieve_Transaction_Rev>:e1) - User:u1 -(User_Recieve_Transaction>:e2) -Transaction:x -(User_Transfer_Transaction_Rev>:e3) - User:u2
  
          WHERE u1.trust_score > 0.5 AND u2.trust_score > 0.5 AND t.ts > timeStamp
          ACCUM
            t.@receiverSet = u1,
            t.@receiverTrust = u1.trust_score,
            t.@senderSet = u2,
            t.@senderTrust = u2.trust_score
          
          ORDER BY t.ts ASC
          LIMIT 1;
  ELSE IF sortValue == "" THEN
  
    s2 = SELECT t FROM s1:t -((User_Recieve_Transaction_Rev>|User_Transfer_Transaction_Rev>):e) - User:u
      WHERE t.ts > timeStamp
      ACCUM
          case when e.type == "User_Recieve_Transaction_Rev" then
              t.@receiverSet += u,
              t.@receiverTrust += u.trust_score
          ELSE
              t.@senderSet += u,
              t.@senderTrust += u.trust_score
          END
     ORDER BY t.ts ASC
      LIMIT 1;
  
  ELSE
    s2 = SELECT t FROM s1:t -((User_Recieve_Transaction_Rev>|User_Transfer_Transaction_Rev>):e) - User:u
      WHERE t.ts > timeStamp
      ACCUM
          case when e.type == "User_Recieve_Transaction_Rev" then
              t.@receiverSet += u,
              t.@receiverTrust += u.trust_score
          ELSE
              t.@senderSet += u,
              t.@senderTrust += u.trust_score
          END
     ORDER BY t.ts ASC
      LIMIT 1;
  
  
  END;
  PRINT s2; 
}
INSTALL Query GetMostRecentTransaction
''', options=[]))

In [ ]:
# Create query to grab transactions to fill data table
print(conn.gsql('''
USE GRAPH AntiFraud
CREATE QUERY GetRecentTransactionID(STRING startDate = "2017-01-15", STRING endDate = "2017-04-15", INT startingIndex = 0, INT pageSize = 25, STRING sortValue="") FOR GRAPH AntiFraud syntax v2{ 
  
  
  ListAccum<VERTEX> @receiverSet, @senderSet;
  SumAccum<FLoat> @receiverTrust, @senderTrust;
  
  Seed = {Day.*};
  
  IF sortValue == ">50" THEN
      s1 = SELECT t FROM Seed:d -(DAY_TO_TRANSACTION:e) -:t
           WHERE d.dateValue < to_datetime(endDate) AND d.dateValue > to_datetime(startDate) AND t.amount > 50
           ORDER BY t.ts DESC
           LIMIT pageSize OFFSET startingIndex;
  ELSE IF sortValue == "<=50" THEN
      s1 = SELECT t FROM Seed:d -(DAY_TO_TRANSACTION:e) -:t
           WHERE d.dateValue < to_datetime(endDate) AND d.dateValue > to_datetime(startDate) AND t.amount < 50
           ORDER BY t.ts DESC
           LIMIT pageSize OFFSET startingIndex;
  ELSE 
      s1 = SELECT t FROM Seed:d -(DAY_TO_TRANSACTION:e) -:t
           WHERE d.dateValue < to_datetime(endDate) AND d.dateValue > to_datetime(startDate)
           ORDER BY t.ts DESC;
  END;
  
  IF sortValue == "<=.5" THEN
      s2 = SELECT t FROM s1:t - (User_Recieve_Transaction_Rev>:e1) - User:u1 -(User_Recieve_Transaction>:e2) -Transaction:x -(User_Transfer_Transaction_Rev>:e3) - User:u2
  
          WHERE u1.trust_score < 0.5 AND u2.trust_score < 0.5
          ACCUM
            t.@receiverSet = u1,
            t.@receiverTrust = u1.trust_score,
            t.@senderSet = u2,
            t.@senderTrust = u2.trust_score
          
          ORDER BY t.ts DESC
          LIMIT pageSize OFFSET startingIndex;
  ELSE
  
      s2 = SELECT t FROM s1:t - (User_Recieve_Transaction_Rev>:e1) - User:u1 -(User_Recieve_Transaction>:e2) -Transaction:x -(User_Transfer_Transaction_Rev>:e3) - User:u2
  
          WHERE u1.trust_score > 0.5 AND u2.trust_score > 0.5
          ACCUM
            t.@receiverSet = u1,
            t.@receiverTrust = u1.trust_score,
            t.@senderSet = u2,
            t.@senderTrust = u2.trust_score
          
          ORDER BY t.ts DESC
          LIMIT pageSize OFFSET startingIndex;
  
  END;
  PRINT s2; 
}
INSTALL Query GetRecentTransactionID
''', options=[]))

In [ ]:
# Returns average credit card risk for all cards tied to a given user
print(conn.gsql('''
USE GRAPH AntiFraud
CREATE QUERY PaymentRisk(Vertex<User> Source) FOR GRAPH AntiFraud { 
  AvgAccum @@avgRisk;
  start = {Source};
  
  card_risk = SELECT tgt
                FROM start:s -(User_to_Payment) -:tgt
                ACCUM @@avgRisk += tgt.trust_score;
  
  PRINT @@avgRisk AS avgRisk;
}
INSTALL QUERY PaymentRisk
''', options=[]))

In [ ]:
# Returns all transactions involving a given user 
print(conn.gsql('''
USE GRAPH AntiFraud
CREATE QUERY TotalTransaction(Vertex<User> Source) FOR GRAPH AntiFraud {  
  start = {Source};
  
  transfer = SELECT tgt
                    FROM start:s -(User_Transfer_Transaction:e) - :tgt;
  
  receive = select tgt
                FROM start:s -(User_Recieve_Transaction:e) -:tgt;

  PRINT transfer, receive;
}
INSTALL QUERY TotalTransaction
''', options=[]))

## App

### Navbar

In [41]:
TG_LOGO = "https://media.glassdoor.com/sqll/1145722/tigergraph-squarelogo-1544825603428.png"
FONT_AWESOME = "https://use.fontawesome.com/releases/v5.8.1/css/all.css"

# Creates search bar to be placed in navbar
search_bar = dbc.Row(
    [
        dbc.Col(dbc.Input(type="search", placeholder="Search", id="search_bar")),
        dbc.Col(
            dbc.Button("Search", color="primary", className="ml-2", id="search_bar_button"),
            width="auto",
        ),
    ], 
    style={"margin-right": "3rem"},
    no_gutters=True,
    className="ml-auto flex-nowrap mt-3 mt-md-0",
    align="center",
)

# Creates a navbar
navbar = dbc.Navbar(
    [
        html.A(
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [
                    dbc.Col(html.Img(src=TG_LOGO, height="30px")),
                    dbc.Col(dbc.NavbarBrand("Anti Fraud", className="ml-2")),
                ], style={"margin-left": "11%"},
                align="center",
                no_gutters=True,
            ),
            href="https://tigerGraph.com",
        ),
        dbc.NavbarToggler(id="navbar-toggler"),
        dbc.Collapse(search_bar, id="navbar-collapse", navbar=True),
    ], 
    color="dark",
    dark=True,
    sticky="top",
    style={'width': 'calc(100% - 12rem)', 'float': 'right', 'height': '4.5rem'}
)

### Sidebar

In [42]:
# CSS styling for sidebar
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "12rem",
    "padding": "1rem 1rem",
    "background-color": "#f8f9fa",
    'text-align': 'center'
}

# CSS styling for main-page contents (user page and transaction page)
CONTENT_STYLE = {
    "margin-left": "13rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

# Links for page navigation
links = {
    "User Transactions": ["/user-transactions", "user-transactions-link"],
    "Global Transactions": ["/global-transactions", "global-transactions-link"]
}

# Creates a sidebar with links and styling above
sidebar = html.Div(
    [
        html.H2("Pages", className="display-6"),
        html.Hr(),
        dbc.Nav(
            [dbc.NavLink(x, href=links[x][0], id=links[x][1]) for x in links.keys()],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

### User page

In [43]:
# Empty placeholder charts used in case some error occurs
initial_table = go.Figure(data=[go.Table(header=dict(values=['Date', 'Amount']), cells=dict())])
initial_dial = go.Figure(go.Indicator(mode='gauge+number',value=0,title = {'text': "Risk for User"}, domain = {'x': [0, 1], 'y': [0, 1]}, gauge = {'axis': {'range': [None, 1]}}), layout={"height":250})

# Layout for user info page
# Page wrapped in dcc.Loading widget to add a loading animation when the page is loaded/updated
user_page = dcc.Loading(
    children=html.Div([
        dbc.Row([
            dbc.Col(html.Div(
                dcc.Graph(id='risk_dial_user', figure=initial_dial),
                style={"background-color": "#424242", 'width': '100%', 'height': '100%'},
            ),
            width=4,
            ),
        dbc.Col(html.Div(
                dcc.Graph(id='risk_dial_card', figure=initial_dial),
                style={"background-color": "#424242", 'width': '100%', 'height': '100%'},
            ),
            width=4,
            ),
        dbc.Col(html.Div(
                dcc.Graph(id='bullet_chart_data', figure=initial_table),
                style={"background-color": "#424242", 'width': '100%', 'height': '100%'},
            ),
            width=4,
            ),
        ], style={"height": "200px", "width": "100%", "margin-left": "auto", "margin-right": "auto", "margin-bottom": "10px", "margin-top": "10px"}),
        dbc.Row(dbc.Col(html.Div(
                    dcc.Graph(id='bar_chart', figure=px.bar(height=350, labels={'x': "Date", "y": "Amount"})),
                    style={"background-color": "#424242", 'width': '100%', 'height': '100%', "vertical-align": "center"}               
                ),
                width=12,
                ),
                style={"height": "350px", "width": "100%", "margin-left": "auto", "margin-right": "auto", "margin-bottom": "10px"}
        ),
        dbc.Row(dbc.Col(html.Div(
                    dcc.Graph(id='table_chart', figure=initial_table),
                    style={"background-color": "#424242", 'width': '100%', 'height': '100%'},   
                ),
                width=12,
                ),
                style={"height": "250px", "width": "100%", "margin-left": "auto", "margin-right": "auto"}
        )
    ]), type='default', id='user_loading'
)

### User callback Function

In [44]:
# Callback to grab user data
def getCharts(userID):
    dates = []
    amounts = []
    transfer_type = []

    # Grab transactions for given user
    results = conn.runInstalledQuery("TotalTransaction", params={"Source": userID})[0]

    # Create table of transactions
    for action in results:
        for transfer in results[action]:
            dates.append(datetime.datetime.fromtimestamp(transfer['attributes']['ts']))
            amounts.append(transfer['attributes']['amount'])
            transfer_type.append(action)
    cols = list(zip(dates, amounts, transfer_type))
    cols = sorted(cols, key=lambda x: x[0].day)
    cols = sorted(cols, key=lambda x: x[0].month)
    cols = sorted(cols, key=lambda x: x[0].year)
    df = pd.DataFrame(data=cols, columns=['Date', 'Amount', 'Type'])
    df['Date'] = pd.to_datetime(df['Date'])
    map_color = {"receive":"rgba(0,0,255,0.5)", "transfer":"rgba(255,0,0,0.5)"}
    df['Color'] = df['Type'].map(map_color)
    table = go.Figure(data=[
        go.Table(header=dict(values=list([col for col in df.columns if col != 'Color'])),
        cells=dict(
            values=[df.Date.dt.date, df.Amount, df.Type], 
            align='left',
            fill_color = [df.Color]
            ))
    ])

    # Create bar chart of transactions
    df = df.groupby([df['Date'].dt.to_period('M'), 'Type', 'Color']).sum()
    df = df.reset_index(level=['Type', 'Color'])
    df.index = df.index.values.astype('datetime64[M]')
    bar = px.bar(df, x=df.index, y='Amount', labels={'x': 'Date'}, color='Type', color_discrete_map = map_color, text='Amount', title="Transaction Amounts by Month", height=350, barmode='group')
    bar.update_xaxes(
        dtick="M1",
        tickformat="%b\n%Y")
    for trace in bar.data:
        trace.name = trace.name.split('=')[1].capitalize()

    # Get average card risk score for given user
    risk_score = conn.getVerticesById(vertexIds=userID, vertexType="User")[0]['attributes']['trust_score']

    # Create user risk dial
    user_dial = go.Figure(go.Indicator(
        mode='gauge+number',
        value=risk_score,
        title = {'text': f"User {userID} Trust Score"},
        domain = {'x': [0, 1], 'y': [0, 1]},
        gauge = {
            'axis': {'range': [None, 1]},
            'bar': {'color': 'rgba(0,0,0,0)'},
             'steps': [
                {'range': [0, 0.3], 'color': 'red'},
                {'range': [0.3, 0.7], 'color': 'orange'},
                {'range': [0.7, 1.0], 'color': 'green'},
                ],
            'threshold': {
                'line': {'color': "black", 'width': 4},
                'thickness': 0.75,
                'value': risk_score}
        }
    ), layout={"height":275})

    # Get average card risk score for user from query
    card_risk_score = conn.runInstalledQuery("PaymentRisk", params={"Source": userID})[0]['avgRisk']

    # Create card risk score dial
    card_dial = go.Figure(go.Indicator(
        mode='gauge+number',
        value=card_risk_score,
        title = {'text': f"Avg Card Trust Score for User {userID}"},
        domain = {'x': [0, 1], 'y': [0, 1]},
        gauge = {
            'axis': {'range': [None, 1]},
            'bar': {'color': 'rgba(0,0,0,0)'},
             'steps': [
                {'range': [0, 0.3], 'color': 'red'},
                {'range': [0.3, 0.7], 'color': 'orange'},
                {'range': [0.7, 1.0], 'color': 'green'},
                ],
            'threshold': {
                'line': {'color': "black", 'width': 4},
                'thickness': 0.75,
                'value': card_risk_score}
        }
    ), layout={"height":275})

    # Create bullet chart of macro stats of user
    total_transfers = len(transfer_type)
    amount_sent = 0
    amount_received = 0
    for i in range(len(transfer_type)):
        if transfer_type[i] == "receive":
            amount_received += amounts[i]
        else:
            amount_sent += amounts[i]

    bullet = go.Figure()
    bullet.add_trace(go.Indicator(
        mode = "number+gauge",
        value = amount_sent,
        domain = {'x': [0.4, 1], 'y': [0.8, 1.0]},
        title = {'text': 'Amount Sent'},
        gauge = {
            'shape': "bullet",
            'axis': {'range': [None, amount_sent + 100]},
            'steps': [
                {'range': [0, (amount_sent + 100)/3], 'color': 'gray'},
                {'range': [(amount_sent + 100)/3, 2*(amount_sent + 100)/3], 'color': 'lightgray'},
            ],
            'bar': {'color': 'black'}
        }
    ))
    bullet.add_trace(go.Indicator(
        mode = "number+gauge",
        value = amount_received,
        domain = {'x': [0.4, 1], 'y': [0.4, 0.65]},
        title = {'text': 'Amount Received'},
        gauge = {
            'shape': "bullet",
            'axis': {'range': [None, amount_received + 100]},
            'steps': [
                {'range': [0, (amount_received + 100)/3], 'color': 'gray'},
                {'range': [(amount_received + 100)/3, 2*(amount_received + 100)/3], 'color': 'lightgray'},
            ],
            'bar': {'color': 'black'}
        }
    ))
    bullet.add_trace(go.Indicator(
        mode = "number+gauge",
        value = total_transfers,
        domain = {'x': [0.4, 1], 'y': [0.1, 0.25]},
        title = {'text': '# Transfers'},
        gauge = {
            'shape': "bullet",
            'axis': {'range': [None, total_transfers + 20]},
            'steps': [
                {'range': [0, (total_transfers + 20)/3], 'color': 'gray'},
                {'range': [(total_transfers + 20)/3, 2*(total_transfers + 20)/3], 'color': 'lightgray'},
            ],
            'bar': {'color': 'black'}
        }
    ))
    bullet.update_layout(height=275, margin={'t': 50})

    # Return all charts 
    return bar, table, user_dial, card_dial, bullet

### Transaction grab 

In [45]:
# Adds flags to suspicious transactions
def flagTransaction(receiverScore, senderScore, transString):
    if receiverScore < 0.3 and senderScore < 0.3:
        transString = "\U0001F6A9 " + transString
    return transString

# Gets data for transaction page
def getTransactions(startDate=None, endDate=None, page=0, sortValue=None):
    
    # Set number of values per page in table
    pageSizeValue = 25

    # Run query
    if startDate is not None and endDate is not None:
        if sortValue:
            x = conn.runInstalledQuery("GetRecentTransactionID", params={'startDate': startDate, 'endDate': endDate, 'startingIndex': page*pageSizeValue , 'pageSize' : pageSizeValue, 'sortValue': sortValue}, sizeLimit=100000000)
        else:
            x = conn.runInstalledQuery("GetRecentTransactionID", params={'startDate': startDate, 'endDate': endDate, 'startingIndex': page*pageSizeValue , 'pageSize' : pageSizeValue}, sizeLimit=100000000)
    elif sortValue:
        x = conn.runInstalledQuery("GetRecentTransactionID", params = {'startingIndex': page*pageSizeValue , 'pageSize': pageSizeValue, 'sortValue': sortValue}, sizeLimit=1000000000)
    else:
        x = conn.runInstalledQuery("GetRecentTransactionID", params={'startingIndex': page*pageSizeValue , 'pageSize': pageSizeValue}, sizeLimit=1000000000)
    
    # Clean up output data
    try:   
        df_flat = pd.json_normalize(x[0]['s2'])
        if len(x[0]['s2']) == 0:
            return df_flat
        df_flat.rename(columns={'v_id': 'Transaction ID', 'attributes.ts': 'Time Stamp', 'attributes.amount': 'Amount', 
                                'attributes.@receiverTrust': 'Receiver Trust Score', 'attributes.@receiverSet': 'Receiver ID', 
                                'attributes.@senderSet': 'Sender ID', 'attributes.@senderTrust': 'Sender Trust Score'}, inplace=True)
        df_flat['Receiver ID'] = df_flat['Receiver ID'].str[0]
        df_flat['Sender ID'] = df_flat['Sender ID'].str[0]
        df_flat.drop('v_type', 1, inplace=True)
        df_flat['Time Stamp'] = df_flat['Time Stamp'].swifter.apply(lambda x: datetime.datetime.fromtimestamp(x))
    except Exception as e:
        print(e)
    
    try:
        df_flat['Transaction ID'] = df_flat.swifter.apply(lambda x: flagTransaction(x['Receiver Trust Score'], x['Sender Trust Score'], x['Transaction ID']), axis=1)
    except Exception as e:
        print(e)
    
    return df_flat

# Gets data to update transaction page (live feature)
def getRecentTransaction(timeStamp, dateStamp, sortValue = None):

    # Run query
    if sortValue:
        x = conn.runInstalledQuery("GetMostRecentTransaction", params={'timeStamp': timeStamp, 'dateStamp': dateStamp, 'sortValue': sortValue})
    else:
        x = conn.runInstalledQuery("GetMostRecentTransaction", params={'timeStamp': timeStamp, 'dateStamp': dateStamp})
    
    returnTime = x[0]['s2'][0]['attributes']['ts']
    
    # Clean up output
    try:
        df_flat = pd.json_normalize(x[0]['s2'])
        if len(x[0]['s2']) == 0:
            return df_flat
        df_flat.rename(columns={'v_id': 'Transaction ID', 'attributes.ts': 'Time Stamp', 'attributes.amount': 'Amount', 
                                'attributes.@receiverTrust': 'Receiver Trust Score', 'attributes.@receiverSet': 'Receiver ID', 
                                'attributes.@senderSet': 'Sender ID', 'attributes.@senderTrust': 'Sender Trust Score'}, inplace=True)
        df_flat['Receiver ID'] = df_flat['Receiver ID'].str[0]
        df_flat['Sender ID'] = df_flat['Sender ID'].str[0]
        df_flat.drop('v_type', 1, inplace=True)
        df_flat['Time Stamp'] = df_flat['Time Stamp'].swifter.apply(lambda x: datetime.datetime.fromtimestamp(x))
        df_flat['Transaction ID'] = df_flat.swifter.apply(lambda x: flagTransaction(x['Receiver Trust Score'], x['Sender Trust Score'], x['Transaction ID']), axis=1)
        returnDate = df_flat['Time Stamp'].iloc[0].date().strftime('%Y-%m-%d')
    except Exception as e:
        print(e)
    return df_flat, returnTime, returnDate

### Transaction page

In [46]:
# Set columns for data table
columns = ['Transaction ID', 'Time Stamp', 'Amount', 'Receiver Trust Score', 'Receiver ID', 'Sender ID', 'Sender Trust Score']

# Create page for global transaction information
# Download component allows for downloading the data table
transaction_page = html.Div([
    Download(id="download"),
    dbc.Row([
           dbc.Col(html.H1('Transaction Analysis'), width=4),
           dbc.Col(daq.BooleanSwitch(id='live_switch', on=False, label="Live", labelPosition="bottom"), width=1)
    ], justify='between'),
    dbc.Row([
        dbc.Col(
            # Date picker to filter table into date range
            dcc.DatePickerRange(
                id='date-picker-range',
                min_date_allowed=datetime.date(1970, 1, 1),
                max_date_allowed=datetime.date(2020, 10, 5),
                initial_visible_month=datetime.date(2017, 4, 30),
                end_date=datetime.date(2017, 5, 15),
                style={'position':'relative', 'zIndex':'999'}
            ),
            width=3
        ),
        dbc.Col(
            # Download button that allows the table to be downloaded
            dbc.Button(
                id='download_button',
                children=html.Span(["Download Results  ", html.I(className="fa fa-download fa-lg")]),
                color='primary',
                disabled=True,
            ),
            width=2,
        ),
        dbc.Col(
            # Dropdown for adding filters to table
            dcc.Dropdown(
                id='filter_dropdown',
                placeholder="Filter Data",
                options=[
                    {'label': 'Fraud Score <= .5', 'value': '<=.5'},
                    {'label': 'Fraud Score > .5', 'value': '>.5'},
                    {'label': 'Transaction Amount <= 50', 'value': '<=50'},
                    {'label': 'Transaction Amount > 50', 'value': '>50'}
                ],
            ),
            width=3
        )
    ], justify='between'), 
    # Data table
    html.Div(id='transaction_table_output', 
            children=dash_table.DataTable(
                id='transaction_table',
                fixed_rows={'headers': True, 'data': 0},
                style_table={'height': '500px'},
                style_cell={'whiteSpace': 'normal'},
                css=[{'selector': '.row', 'rule': 'margin: 0'}],
                columns= [{"name": i, "id": i} for i in columns],
                style_data_conditional=[{'if': {'column_id': f'{i}'}, 'width': '50px'} for i in columns],
                page_current=0,
                page_size=25,
                page_action='custom'), 
        style={'margin-top': '2em'}
    )   
], style={'margin-top': '5em'})

### Run app

In [ ]:
# Creates instance of app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, FONT_AWESOME])

# Global variable used to store results
df = None
timeStamp = 1492214384
dateStamp = "2017-04-15"
live = False

# Set app layout
app.layout = html.Div([
                dcc.Location(id="url", refresh=False),
                dcc.Store(id='userIDStore'),
                dcc.Interval(
                    id='interval-component',
                    interval=7*1000,
                    n_intervals=0,
                    disabled=True
                ),
                sidebar,
                navbar,
                html.Div(id='page-content', style=CONTENT_STYLE),
             ])

# Transitions to user page when you click on user ID in transaction table
@app.callback(
    [Output(component_id='userIDStore', component_property='data'), Output(component_id='url', component_property='pathname')],
    [Input(component_id='transaction_table', component_property='active_cell')],
    [State(component_id='transaction_table', component_property='data')]
)
def getActiveCell(active_cell, data):
    if active_cell:
        col = active_cell['column_id']
        row = active_cell['row']
        if col == 'Receiver ID' or col == 'Sender ID':
            return data[row][col], links[list(links.keys())[0]][0]
    return no_update, no_update

# Toggles the downlad button 
@app.callback(
    Output('download_button', 'disabled'),
    [Input('interval-component', 'disabled')]
)
def allowDownloads(disabled):
    return not disabled

# Toggles interval (live update)
@app.callback(
    [Output(component_id='interval-component', component_property='disabled'), Output(component_id='interval-component', component_property='n_intervals')],
    [Input(component_id='live_switch', component_property='on')],
    [State(component_id='interval-component', component_property='n_intervals')]
)
def turnLive(on, n):
    if on:
        n += 1
    return not on, n

# Updates table data
@app.callback(
    Output('transaction_table', 'data'),
    [Input('interval-component', 'n_intervals'), Input('date-picker-range', 'start_date'), 
     Input('date-picker-range', 'end_date'), Input('filter_dropdown', 'value'), Input('transaction_table', 'page_current')],
    [State('transaction_table', 'data'), State('interval-component', 'disabled')]
)
def updateTable(n_intervals, start_date, end_date, value, page_current, rows, disabled):

    global df, timeStamp, dateStamp, live

    # Default table data
    if ((start_date is None or end_date is None) and value is None and n_intervals == 0):
        rows = getTransactions(page=page_current).to_dict('records')
        df = pd.DataFrame.from_dict(rows)
        return rows
    
    # Update if live feature enabled
    if n_intervals > 0:
        if not disabled:
            live = True
            try:
                if value is not None:
                    x, y, z = getRecentTransaction(timeStamp=timeStamp, dateStamp=dateStamp, sortValue=value)
                else:
                    x, y, z = getRecentTransaction(timeStamp=timeStamp, dateStamp=dateStamp)
                
                d = x.to_dict('records')[0]
                timeStamp = y
                dateStamp = z
                # df = df.append(x)
                if rows:
                    rows.insert(0, d)
                else:
                    rows = [d]
                return rows
            except:
                return no_update
    
    # Update if date range provided
    if start_date is not None and end_date is not None:
        if not live:
            try:
                if value is not None:
                    rows = getTransactions(startDate=start_date, endDate=end_date, page=page_current, sortValue=value).to_dict('records')
                    df = pd.DataFrame.from_dict(rows)
                else:
                    rows = getTransactions(startDate=start_date, endDate=end_date, page=page_current).to_dict('records')
                    df = pd.DataFrame.from_dict(rows)
                
                time = rows[0]['Time Stamp']
                timeStamp = int(datetime.datetime.timestamp(time))
                dateStamp = time.date().strftime('%Y-%m-%d')
                return rows
            except:
                return no_update
        else:
            live = False
            return rows
    
    # Filter data
    if value is not None:
        rows = getTransactions(page=page_current, sortValue=value, endDate=dateStamp).to_dict('records')
        df = pd.DataFrame.from_dict(rows)
        return rows
    
    return rows

# Downloads table
@app.callback(
    Output("download", "data"), 
    [Input("download_button", "n_clicks")],
    [State('transaction_table', 'data')]
)
def generate_csv(n_clicks, data):
    if n_clicks:
        return send_data_frame((pd.DataFrame.from_dict(data)).to_csv, filename="Transactions.csv")
        
# Generate user plots
@app.callback(
    [Output(component_id='bar_chart', component_property='figure'), Output(component_id='table_chart', component_property='figure'), 
     Output(component_id='risk_dial_user', component_property='figure'), Output(component_id='risk_dial_card', component_property='figure'),
     Output(component_id="bullet_chart_data", component_property="figure")],
    [Input(component_id='search_bar_button', component_property='n_clicks'), Input('userIDStore', 'data')],
    [State(component_id='search_bar', component_property='value')]
)
def updateUser(n, storedData, user_id):
    try:
        if storedData is not None:
            return getCharts(storedData)
        if n is None:
            return getCharts("333")
        else:
            return getCharts(user_id)
    except Exception as e:
        return no_update, no_update, no_update, no_update, no_update

# Switch pathname for url
@app.callback(
    [Output(f"{links[id][1]}", "active") for id in links.keys()],
    [Input("url", "pathname")],
)
def toggle_active_links(pathname):
    if pathname == "/" or pathname == "//":
        # Treat page 1 as the homepage / index
        return True, False
    return [pathname == f"{links[id][0]}" for id in links.keys()]

# Set page for each pathname
@app.callback(
    Output("page-content", "children"), 
    [Input("url", "pathname")]
)
def render_page_content(pathname):
    if pathname in ["/", "//", f"{links[list(links.keys())[0]][0]}"]:
        return user_page
    else:
        return transaction_page
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )
if __name__ == '__main__':
    app.run_server(mode='external')